# Vaje 11: Manjkajoče vrednosti in neenakomerne porazdelitve ciljne spremenljivke

## Naloga 1: Manjkajoče vrednosti

V praksi se pogosto spopademo s podatki, ki vsebujejo manjkajoče vrednosti. V tej nalogi si bomo pogledali nekaj pristopov za spopadanje s takšnimi podatki, kot so:
- uporaba napovednih modelov, ki se lahko spopade z njimi (npr. drevesa)
- odstranitev problematičnih stolpcev
- nadomestitev manjkajočih vrednosti v danem stolpcu s povprečjem znanih
- napoved manjkajočih vrednosti v danem stolpcu s pomočjo modela

Vse metode za delo z manjkajočimi vrednostmi vhodnih spremenljivk so primerne tudi za delo s podatki, ki jim manjkajo nekatere ciljne vrednosti, zato bodo nekatere funkcije sposobne odpraviti tudi manjkajoče vrednosti v ciljni spremenljivki (ali pa bi bile za to potrebne le majhne spremembe). Paziti je treba le (če to delamo pred razdelitvijo na učno in testno množico), da vrednosti ciljne spremenljivke iz testne ne uporabljamo.

Na današnjih vajah bomo manjkajoče vrednosti označili z NA. Za zaznavanje NA-jev je koristna funkcija
*is.na* (ter sorodna *anyNA*).


Sestavimo najprej podatkovno množico in jo razdelimo na učno in testno množico

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

p = 0.4
X = np.random.random((100, 10))
y = X @ np.arange(11, 21) + X**2 @ np.arange(20, 10, -1)
for i in range(100):
    if np.random.random() < p:
        X[i, np.random.randint(5, size=2)*2] = np.nan
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

1.a: Najprej uporabimo odločitvena drevesa, ki se znajo brez dodatnih izboljšav spopasti z manjkajočimi podatki. Premisli zakaj je temu tako in preveri točnost odločitvenega drevesa na danih podatkih.

In [2]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [3]:
model = DecisionTreeRegressor().fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"Decision tree accuracy on test set; RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}")

Decision tree accuracy on test set; RMSE: 32.6140467605478


Odločitvena drevesa primere razdelijo glede na to ali izpolnjujejo pogoj ali ne. Če je v podatkih torej manjkajoča vrednost, pogoj ni izpolnjen, torej gre primer v vejo, ki je namenjena primerom z neizpolnjenim pogojem.

1.b: Drugi pristop je, da odstanimo stolpce oziroma vrstice, ki vsebujejo manjkajoče vrednosti. Preveri točnost modela, ki uporablja podatke, ki jim odstanimo stolpce z manjkajočimi vrednostmi oziroma vrstice z manjkajočimi vrednostmi. Pomagaš si lahko s funkcijama `numpy.isnan` in `numpy.any`. Opaziš kakšen problem pri tem pristopu?

In [4]:
row_mask = np.isnan(X_train).any(axis=1)
col_mask = np.isnan(X_train).any(axis=0)
X_rows = X_train[row_mask]
X_cols = X_train[:, col_mask]

row_model = DecisionTreeRegressor().fit(X_rows, y_train[row_mask])
y_pred = row_model.predict(X_test)
print(f"Decision tree without NAN rows accuracy on test set; RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}")

col_model = DecisionTreeRegressor().fit(X_cols, y_train)
y_pred = col_model.predict(X_test[:, col_mask])
print(f"Decision tree without NAN columns accuracy on test set; RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}")

Decision tree without NAN rows accuracy on test set; RMSE: 24.71219380385893
Decision tree without NAN columns accuracy on test set; RMSE: 32.422976403296765


Pri uporabi tega pristopa lahko pride do več problemov. Kaj bomo napovedali za testne primere, ki vsebujejo NAN vrednosti, kaj če se NAN vrednost pojavi v stolpcu, ki v učni množici ni vseboval NAN vrednosti, kaj če vse vrstice/stolpci vsebujejo NAN vrednosti, ...

Odstranjevanje značilk ali primerov je redko dobra izbira. Preprosta in kar dobra alternativa je manjkajočo vrednost zapolniti s povprečjem (aritmetičnim ali mediano) v primeru numeričnih spremenljivk, oziroma z najpogostejšo vrednostjo v primeru kategoričnih spremenljivk.

1.c: Preveri točnost modela, ko manjkajoče vrednosti nadomestiš z aritmetičnim povprečjem in mediano. V praksi je mediana uporabna predvsem, ko imamo "outlierje", ki bi zelo pokvarili povprečje.

In [5]:
mask = np.isnan(X_train)
test_mask = np.isnan(X_test)
mean_values = np.nanmean(X_train, axis=0)
median_values = np.nanmedian(X_train, axis=0)
X_train_mean = X_train.copy()
X_train_median = X_train.copy()
X_test_mean = X_test.copy()
X_test_median = X_test.copy()

for i in range(X_train_mean.shape[1]):
    X_train_mean[mask[:, i], i] = mean_values[i]
    X_train_median[mask[:, i], i] = median_values[i]
    X_test_mean[test_mask[:, i], i] = mean_values[i]
    X_test_median[test_mask[:, i], i] = median_values[i]

mean_model = DecisionTreeRegressor().fit(X_train_mean, y_train)
y_pred = mean_model.predict(X_test_mean)
print(f"Decision tree with mean imputer on test set; RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}")

median_model = DecisionTreeRegressor().fit(X_train_median, y_train)
y_pred = median_model.predict(X_test_median)
print(f"Decision tree with median imputer on test set; RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}")

Decision tree with mean imputer on test set; RMSE: 32.11413818201312
Decision tree with median imputer on test set; RMSE: 32.60701772627695


Najzahtevnejši pristop za obravnavo manjkajočih vrednosti je napovedovanje s pomočjo modela - za to se pogosto uporablja metoda najbližjih sosedov. Takšno imputacijo nam olajša sklearn-ov model KNNImputer. 
POZOR! Pri prejšnjih metodah ni bilo zares pomembno, pri imputaciji z modelom pa je: paziti moramo, da predprocesiranje treniramo na učni množici, njegove vrednosti pa potem uporabimo za predprocesiranje testne množice. 

1.d: Z uporabo objekta KNNImputer dopolni manjkajoče vrednosti v podatkih in preveri točnost odločitvenega drevesa na testni množici.

In [6]:
from sklearn.impute import KNNImputer

In [7]:
imputer = KNNImputer().fit(X_train)
X_train_imputed = imputer.transform(X_train)
X_test_imputed = imputer.transform(X_test)

imputed_model = DecisionTreeRegressor().fit(X_train_imputed, y_train)
y_pred = imputed_model.predict(X_test_imputed)
print(f"Decision tree with KNN imputer on test set; RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}")

Decision tree with KNN imputer on test set; RMSE: 30.867584447653886


## Naloga 2: Neenakomerne porazdelitve ciljne spremenljivke

Kadar je porazdelitev ciljne spremenljivke močno neenakomerna, lahko pri učenju modelov nastopijo težave. Ker bolj točne napovedi za pogosto opazovane vrednosti manjšajo napako, postanejo napovedi pogostih vrednosti ciljne spremenljivke bolj pomembne, in predsodek modela se poveča. Na primer, zelo majhen delež pozitivnih primerov pri dvojiški klasifikaciji lahko vodi do modela, ki vedno napove negativni razred. 

Eden od načinov za spopadanje z neenakomernimi porazdelitvami je previdna ročna konstrukcija funkcije napake, ki da večji poudarek manj zastopanim vrednostim. 

Drug popularen pristop za reševanje teh problem je uteženo vzorčenje podatkov. Ta pristop si bomo pogledali na tej vaji.

Uporabili bomo podatkovno množico, ki vsebuje le okoli 5% pozitivnih primerov.

In [9]:
data = np.load("vaje11.npy", allow_pickle=True)
X = data[:, :-1]
y = data[:, -1].astype(bool)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, stratify=y)

2.a: Podatki vsebujejo manjkajoče vrednosti. Z eno od zgornjih metod te vrednosti dopolni.

In [10]:
X_train[X_train == "?"] = np.nan
X_test[X_test == "?"] = np.nan
X_train = np.array(X_train, dtype=np.float64)
X_test = np.array(X_test, dtype=np.float64)
X_test.astype(np.float64)
mask = np.isnan(X_train)
test_mask = np.isnan(X_test)
median_values = np.nanmedian(X_train, axis=0)

for i in range(X_train.shape[1]):
    X_train[mask[:, i], i] = median_values[i]
    X_test[test_mask[:, i], i] = median_values[i]

Najpreprostejši (in pogosto najboljši) metodi za uteženo vzorčenje sta:

- podvzorčenje (undersampling ali downsampling): vzamemo vse primere manjšinskega razreda ter zgolj delež primerov večinskega razreda, tako da porazdelitev uravnovesimo

- nadvzročenje (oversampling ali upsampling): podatkom dodamo neko število kopij primerov majšinskega razreda, tako da porazdelitev izenačimo

Za uteževa

In [22]:
!pip install imbalanced-learn

2.b: S pomočjo knjižnjice imabalanced-learn sestavi množico z nadvzorčenjem in podvzorčenjem. Preveri confusion matrix za model odločitvenega drevesa naučenega na navadni, nadvzorčeni in podvzorčeni učni množici.

<details>
  <summary>Namig:</summary>

Za nadvzorčenje lahko uporabiš razred [imblearn.over_sampling.RandomOverSampler](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.RandomOverSampler.html), za podvzorčenje pa razred [imblearn.under_sampling.RandomUnderSampler](https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.RandomUnderSampler.html)
   
</details>

In [11]:
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.tree import DecisionTreeClassifier

In [12]:
rus = RandomUnderSampler()
X_rus, y_rus = rus.fit_resample(X_train, y_train)
ros = RandomOverSampler()
X_ros, y_ros = ros.fit_resample(X_train, y_train)

print("Normal data set")
model = DecisionTreeClassifier().fit(X_train, y_train)
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))

print("Undersampled data set")
model = DecisionTreeClassifier().fit(X_rus, y_rus)
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))

print("Oversampled data set")
model = DecisionTreeClassifier().fit(X_ros, y_ros)
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))

Normal data set
[[195   8]
 [  8   0]]
Undersampled data set
[[160  43]
 [  6   2]]
Oversampled data set
[[198   5]
 [  8   0]]


2.c: 

In [13]:
from imblearn.over_sampling import SMOTE

In [14]:
smote = SMOTE()
X_smote, y_smote = smote.fit_resample(X_train, y_train)

print("Data set sampled with SMOTE")
model = DecisionTreeClassifier().fit(X_smote, y_smote)
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))

Data set sampled with SMOTE
[[189  14]
 [  7   1]]


Pomembno: Vzorčenje (posebaj nadvzorčenje) moramo delati na učni množici (pri prečnem preverjanju v vsakem fold-u posebaj). V nasprotnem primeru lahko dodamo primere, ki se pojavijo v učni množici tudi v testno množici in tako umetno izboljšamo točnost